LightGBMモデルによるデフォルト予測

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import xfeat

In [52]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [53]:
#dataframeの表示を省略しない
pd.set_option('display.max_columns', 100)
train.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status
0,0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,FullyPaid
1,1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,ChargedOff
2,2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,FullyPaid
3,3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,FullyPaid
4,4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,FullyPaid


In [54]:
test.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type
0,242150,1161.830205,3 years,13.105362,C1,10 years,credit_card,682.016083,Individual
1,242151,2257.465815,5 years,14.196200,C5,10 years,debt_consolidation,668.240313,Individual
2,242152,2182.137002,3 years,8.307772,A5,10 years,credit_card,703.277694,Individual
3,242153,520.284912,3 years,6.956504,A2,7 years,debt_consolidation,731.138779,Individual
4,242154,2303.349549,3 years,11.658497,B3,6 years,debt_consolidation,698.368991,Individual


数値変数の特徴量エンジニアリング

In [55]:
#termとemployment_lengthからyearを取り除いてint型にする
train["term_int"] = train["term"]
train["employment_length_int"] = train["employment_length"]
test["term_int"] = test["term"]
test["employment_length_int"] = test["employment_length"]


def year_eliminater(dataframe, column1, column2):

    def eliminater(string):
        if string[0:2] == "1 ":
            string = string.replace(' year', '')
        else:
            string = string.replace(' years', '')
        
        return int(string)

    for i in range(len(dataframe)):
        dataframe[column1][i] = eliminater(dataframe[column1][i])
        dataframe[column2][i] = eliminater(dataframe[column2][i])

    return dataframe

train = year_eliminater(train, "term_int", "employment_length_int")
test = year_eliminater(test, "term_int", "employment_length_int")

/var/folders/g3/b4gm8r7d4b7_msz71zk9wg_40000gn/T/ipykernel_38989/2139281269.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column1][i] = eliminater(dataframe[column1][i])
/var/folders/g3/b4gm8r7d4b7_msz71zk9wg_40000gn/T/ipykernel_38989/2139281269.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column2][i] = eliminater(dataframe[column2][i])


In [56]:
#loan_amntとinterest_rateから、最終的な返済額についての列を生成
def calc_total_amount(dataframe):
    total_amount_list = []
    for i in range(len(dataframe)):
        total_amount = dataframe["loan_amnt"][i] * (1 + dataframe["interest_rate"][i]/100) ** dataframe["term_int"][i]
        total_amount_list.append(total_amount)
    dataframe["total_amount"] = total_amount_list
    return dataframe
train = calc_total_amount(train)
test = calc_total_amount(test)

In [57]:
train.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status,term_int,employment_length_int,total_amount
0,0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,FullyPaid,3,0,776.569633
1,1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,ChargedOff,5,10,1930.634031
2,2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,FullyPaid,3,1,1050.582789
3,3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,FullyPaid,3,0,1101.483602
4,4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,FullyPaid,5,10,5055.957652


In [58]:
#列同士の割り算を行うための関数を定義
def calc_division(dataframe, column1, column2, new_column_name):
    divided_num_list = []
    for i in range(len(dataframe)):
        devided_num = dataframe[column1][i] / dataframe[column2][i]
        divided_num_list.append(devided_num)
    dataframe[new_column_name] = divided_num_list
    return dataframe

In [59]:
#各顧客の最終的な返済額と，その顧客のクレジットスコアから，変数"負担指数(burden_indx)"をburden_index = total_amount / credit_scoreとして定義
train = calc_division(train, "total_amount", "credit_score", "burden_index")
test = calc_division(test, "total_amount", "credit_score", "burden_index")

In [60]:
train.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status,term_int,employment_length_int,total_amount,burden_index
0,0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,FullyPaid,3,0,776.569633,1.087538
1,1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,ChargedOff,5,10,1930.634031,2.767114
2,2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,FullyPaid,3,1,1050.582789,1.600475
3,3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,FullyPaid,3,0,1101.483602,1.674224
4,4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,FullyPaid,5,10,5055.957652,7.626197


In [61]:
test.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,term_int,employment_length_int,total_amount,burden_index
0,242150,1161.830205,3 years,13.105362,C1,10 years,credit_card,682.016083,Individual,3,10,1681.094955,2.464890
1,242151,2257.465815,5 years,14.196200,C5,10 years,debt_consolidation,668.240313,Individual,5,10,4384.089811,6.560649
2,242152,2182.137002,3 years,8.307772,A5,10 years,credit_card,703.277694,Individual,3,10,2772.431880,3.942158
3,242153,520.284912,3 years,6.956504,A2,7 years,debt_consolidation,731.138779,Individual,3,7,636.594430,0.870689
4,242154,2303.349549,3 years,11.658497,B3,6 years,debt_consolidation,698.368991,Individual,3,6,3206.529043,4.591454


In [62]:
train.dtypes

id                         int64
loan_amnt                float64
term                      object
interest_rate            float64
grade                     object
employment_length         object
purpose                   object
credit_score             float64
application_type          object
loan_status               object
term_int                  object
employment_length_int     object
total_amount             float64
burden_index             float64
dtype: object

In [63]:
#満了時点での勤続年数
train["final_employment_length"] = train["employment_length_int"] + train["term_int"]
test["final_employment_length"] = test["employment_length_int"] + test["term_int"]

train.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status,term_int,employment_length_int,total_amount,burden_index,final_employment_length
0,0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,FullyPaid,3,0,776.569633,1.087538,3
1,1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,ChargedOff,5,10,1930.634031,2.767114,15
2,2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,FullyPaid,3,1,1050.582789,1.600475,4
3,3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,FullyPaid,3,0,1101.483602,1.674224,3
4,4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,FullyPaid,5,10,5055.957652,7.626197,15


In [64]:
#EDAから．credit_score, interest_rate, loan_amntには５つの山があることがわかる．そこで，上記３変数について，５つのグループのどこに所属するかを表す新たな変数を作成．

def split_into_groups(dataset):
    #credit_scoreを分ける
    dataset.loc[dataset['credit_score'] < 675, 'credit_group'] = 0
    dataset.loc[(dataset['credit_score'] >= 675) & (dataset["credit_score"] < 695), 'credit_group'] = 1
    dataset.loc[(dataset['credit_score'] >= 695) & (dataset["credit_score"] < 710), 'credit_group'] = 2
    dataset.loc[(dataset['credit_score'] >= 710) & (dataset["credit_score"] < 740), 'credit_group'] = 3
    dataset.loc[dataset['credit_score'] >= 740, 'credit_group'] = 4


    #interest_rateを分ける
    dataset.loc[dataset['interest_rate'] < 9.5, 'interest_group'] = 0
    dataset.loc[(dataset['interest_rate'] >= 9.5) & (dataset["interest_rate"] < 12.3), 'interest_group'] = 1
    dataset.loc[(dataset['interest_rate'] >= 12.3) & (dataset["interest_rate"] <= 15.8), 'interest_group'] = 2
    dataset.loc[(dataset['interest_rate'] >= 15.8) & (dataset["interest_rate"] < 20), 'interest_group'] = 3
    dataset.loc[dataset['interest_rate'] >= 20, 'interest_group'] = 4


    #loan_amountを分ける
    dataset.loc[dataset['loan_amnt'] < 850, 'loan_group'] = 0
    dataset.loc[(dataset['loan_amnt'] >= 850) & (dataset["loan_amnt"] < 1350), 'loan_group'] = 1
    dataset.loc[(dataset['loan_amnt'] >= 1350) & (dataset["loan_amnt"] <= 1900), 'loan_group'] = 2
    dataset.loc[(dataset['loan_amnt'] >= 1900) & (dataset["loan_amnt"] < 2700), 'loan_group'] = 3
    dataset.loc[dataset['loan_amnt'] >= 2700, 'loan_group'] = 4

split_into_groups(train)
split_into_groups(test)

In [65]:
train.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status,term_int,employment_length_int,total_amount,burden_index,final_employment_length,credit_group,interest_group,loan_group
0,0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,FullyPaid,3,0,776.569633,1.087538,3,3.0,0.0,0.0
1,1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,ChargedOff,5,10,1930.634031,2.767114,15,2.0,1.0,1.0
2,2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,FullyPaid,3,1,1050.582789,1.600475,4,0.0,2.0,0.0
3,3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,FullyPaid,3,0,1101.483602,1.674224,3,0.0,2.0,0.0
4,4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,FullyPaid,5,10,5055.957652,7.626197,15,0.0,4.0,2.0


カテゴリカル変数のエンコーディング

In [66]:
#まずは目的変数を0,1化
train.replace({'FullyPaid': 0, 'ChargedOff': 1}, inplace=True)

#gradeをA ~ Fの６値にした新たな特徴量grade_numを定義(例： A1 → A)
def grade_translator(dataframe):
    #grade_dict = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5}
    grade_dict = {"A": "A", "B": "B", "C": "C", "D": "D", "E": "E", "F": "F"}
    grade_num_list = []
    for i in range(len(dataframe)):
        grade_num = grade_dict[dataframe["grade"][i][0]]
        grade_num_list.append(grade_num)
    dataframe["grade_num"] = grade_num_list

grade_translator(train)
grade_translator(test)

# #gradeをA ~ Fの６値にして、それぞれ0〜5に変換(例:A1 → A)，新たな特徴量"grade_num"とする．
# def grade_translator(dataframe):
#     grade_dict = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5}
#     grade_num_list = []
#     for i in range(len(dataframe)):
#         grade_num = grade_dict[dataframe["grade"][i][0]]
#         grade_num_list.append(grade_num)
#     dataframe["grade_num"] = grade_num_list

# grade_translator(train)
# grade_translator(test)

In [67]:
train.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status,term_int,employment_length_int,total_amount,burden_index,final_employment_length,credit_group,interest_group,loan_group,grade_num
0,0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,0,3,0,776.569633,1.087538,3,3.0,0.0,0.0,A
1,1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,1,5,10,1930.634031,2.767114,15,2.0,1.0,1.0,B
2,2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,0,3,1,1050.582789,1.600475,4,0.0,2.0,0.0,C
3,3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,0,3,0,1101.483602,1.674224,3,0.0,2.0,0.0,C
4,4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,0,5,10,5055.957652,7.626197,15,0.0,4.0,2.0,E


変数"grade_num"と"employment_length"の，変数"credit_score"および"burden_index"についての集約統計量を新たな特徴量として定義．

In [68]:
#aggregationを行う．
train, aggregated_cols = xfeat.aggregation(train,
                     group_key="grade_num",
                     group_values=["credit_score", "burden_index"],
                     agg_methods=["mean"],
                     )

test, aggregated_cols = xfeat.aggregation(test,
                     group_key="grade_num",
                     group_values=["credit_score", "burden_index"],
                     agg_methods=["mean"],
                     )

train, aggregated_cols = xfeat.aggregation(train,
                     group_key="employment_length",
                     group_values=["credit_score", "burden_index"],
                     agg_methods=["mean"],
                     )

test, aggregated_cols = xfeat.aggregation(test,
                     group_key="employment_length",
                     group_values=["credit_score", "burden_index"],
                     agg_methods=["mean"],
                     )
                     
train.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status,term_int,employment_length_int,total_amount,burden_index,final_employment_length,credit_group,interest_group,loan_group,grade_num,agg_mean_credit_score_grpby_grade_num,agg_mean_burden_index_grpby_grade_num,agg_mean_credit_score_grpby_employment_length,agg_mean_burden_index_grpby_employment_length
0,0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,0,3,0,776.569633,1.087538,3,3.0,0.0,0.0,A,716.496754,3.176289,677.708997,3.669962
1,1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,1,5,10,1930.634031,2.767114,15,2.0,1.0,1.0,B,683.070367,3.049796,684.668141,4.051284
2,2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,0,3,1,1050.582789,1.600475,4,0.0,2.0,0.0,C,674.859374,3.659903,684.394819,3.172965
3,3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,0,3,0,1101.483602,1.674224,3,0.0,2.0,0.0,C,674.859374,3.659903,677.708997,3.669962
4,4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,0,5,10,5055.957652,7.626197,15,0.0,4.0,2.0,E,668.965631,7.497618,684.668141,4.051284


In [69]:
#カテゴリカル変数の組み合わせの特徴量を作成
def combine_categorical(dataframe, column1, column2):
    dataframe[f"{column1}_{column2}"] = dataframe[column1].astype(str) + "_" + dataframe[column2].astype(str)

combine_categorical(train, "employment_length", "purpose")
combine_categorical(test, "employment_length", "purpose")

combine_categorical(train, "employment_length", "term")
combine_categorical(test, "employment_length", "term")

combine_categorical(train, "purpose", "term")
combine_categorical(test, "purpose", "term")

combine_categorical(train, "grade_num", "term")
combine_categorical(test, "grade_num", "term")

combine_categorical(train, "grade_num", "purpose")
combine_categorical(test, "grade_num", "purpose")

combine_categorical(train, "grade_num", "employment_length")
combine_categorical(test, "grade_num", "employment_length")

combine_categorical(train, "grade_num", "employment_length")
combine_categorical(test, "grade_num", "employment_length")

combine_categorical(train, "credit_group", "interest_group")
combine_categorical(test, "credit_group", "interest_group")

combine_categorical(train, "credit_group", "loan_group")
combine_categorical(test, "credit_group", "loan_group")

combine_categorical(train, "interest_group", "loan_group")
combine_categorical(test, "interest_group", "loan_group")

train.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status,term_int,employment_length_int,total_amount,burden_index,final_employment_length,credit_group,interest_group,loan_group,grade_num,agg_mean_credit_score_grpby_grade_num,agg_mean_burden_index_grpby_grade_num,agg_mean_credit_score_grpby_employment_length,agg_mean_burden_index_grpby_employment_length,employment_length_purpose,employment_length_term,purpose_term,grade_num_term,grade_num_purpose,grade_num_employment_length,credit_group_interest_group,credit_group_loan_group,interest_group_loan_group
0,0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,0,3,0,776.569633,1.087538,3,3.0,0.0,0.0,A,716.496754,3.176289,677.708997,3.669962,0 years_debt_consolidation,0 years_3 years,debt_consolidation_3 years,A_3 years,A_debt_consolidation,A_0 years,3.0_0.0,3.0_0.0,0.0_0.0
1,1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,1,5,10,1930.634031,2.767114,15,2.0,1.0,1.0,B,683.070367,3.049796,684.668141,4.051284,10 years_credit_card,10 years_5 years,credit_card_5 years,B_5 years,B_credit_card,B_10 years,2.0_1.0,2.0_1.0,1.0_1.0
2,2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,0,3,1,1050.582789,1.600475,4,0.0,2.0,0.0,C,674.859374,3.659903,684.394819,3.172965,1 year_debt_consolidation,1 year_3 years,debt_consolidation_3 years,C_3 years,C_debt_consolidation,C_1 year,0.0_2.0,0.0_0.0,2.0_0.0
3,3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,0,3,0,1101.483602,1.674224,3,0.0,2.0,0.0,C,674.859374,3.659903,677.708997,3.669962,0 years_credit_card,0 years_3 years,credit_card_3 years,C_3 years,C_credit_card,C_0 years,0.0_2.0,0.0_0.0,2.0_0.0
4,4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,0,5,10,5055.957652,7.626197,15,0.0,4.0,2.0,E,668.965631,7.497618,684.668141,4.051284,10 years_debt_consolidation,10 years_5 years,debt_consolidation_5 years,E_5 years,E_debt_consolidation,E_10 years,0.0_4.0,0.0_2.0,4.0_2.0


Traget Encoding(対象：'employment_length_purpose', 'employment_length_term', 'purpose_term', 'grade_num_term', 'grade_num_purpose', 'grade_num_employment_length', 'grade_num', 'credit_group_interest_group', 'credit_group_loan_group', 'interest_group_loan_group'])

In [70]:
def my_target_encoder(train_df, test_df, column_list):
    fold = KFold(n_splits=5, shuffle=False)
    encoder = xfeat.TargetEncoder(
        input_cols=column_list, 
        target_col="loan_status",
        fold=fold,
        output_suffix="_re"
        )

    encoded_train = encoder.fit_transform(train_df)
    encoded_test = encoder.transform(test_df)

    return encoded_train, encoded_test
    

target_encode_columns = ['employment_length_purpose', 'employment_length_term', 'purpose_term', 'grade_num_term', 'grade_num_purpose', 'grade_num_employment_length', 'grade_num', 'credit_group_interest_group', 'credit_group_loan_group', 'interest_group_loan_group']
train, test = my_target_encoder(train, test, target_encode_columns)
train.drop(columns = target_encode_columns, inplace=True)
test.drop(columns = target_encode_columns, inplace=True)

In [71]:
train.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status,term_int,employment_length_int,total_amount,burden_index,final_employment_length,credit_group,interest_group,loan_group,agg_mean_credit_score_grpby_grade_num,agg_mean_burden_index_grpby_grade_num,agg_mean_credit_score_grpby_employment_length,agg_mean_burden_index_grpby_employment_length,employment_length_purpose_re,employment_length_term_re,purpose_term_re,grade_num_term_re,grade_num_purpose_re,grade_num_employment_length_re,grade_num_re,credit_group_interest_group_re,credit_group_loan_group_re,interest_group_loan_group_re
0,0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,0,3,0,776.569633,1.087538,3,3.0,0.0,0.0,716.496754,3.176289,677.708997,3.669962,0.214557,0.168913,0.148655,0.032232,0.094642,0.096203,0.080336,0.030183,0.053875,0.039061
1,1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,1,5,10,1930.634031,2.767114,15,2.0,1.0,1.0,683.070367,3.049796,684.668141,4.051284,0.143652,0.312443,0.238716,0.254995,0.121464,0.141902,0.136438,0.129636,0.137194,0.132685
2,2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,0,3,1,1050.582789,1.600475,4,0.0,2.0,0.0,674.859374,3.659903,684.394819,3.172965,0.169525,0.136002,0.148655,0.190467,0.211596,0.210969,0.203443,0.178486,0.154018,0.157339
3,3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,0,3,0,1101.483602,1.674224,3,0.0,2.0,0.0,674.859374,3.659903,677.708997,3.669962,0.171772,0.168913,0.117253,0.190467,0.184981,0.219450,0.203443,0.178486,0.154018,0.157339
4,4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,0,5,10,5055.957652,7.626197,15,0.0,4.0,2.0,668.965631,7.497618,684.668141,4.051284,0.198372,0.312443,0.308244,0.318968,0.323903,0.330475,0.309276,0.463604,0.259822,0.448159


In [72]:
#残った変数についてもエンコーディング
train.replace({'Individual': 0, 'Joint App': 1}, inplace=True)
train = pd.get_dummies(train, prefix='', prefix_sep='', columns=['term', 'grade', 'purpose'])

In [73]:
#application_typeについてもエンコーディング
test.replace({'Individual': 0, 'Joint App': 1}, inplace=True)
test = pd.get_dummies(test, prefix='', prefix_sep='', columns=['term', 'grade', 'purpose'])

In [80]:
#trainデータとtestデータで説明変数を揃えるための処理
test[list(set(train.columns).difference(set(test.columns)))] = 0
train[list(set(test.columns).difference(set(train.columns)))] = 0

test.drop(columns=["loan_status"], inplace=True)

In [81]:
train.head()

,id,loan_amnt,interest_rate,employment_length,credit_score,application_type,loan_status,term_int,employment_length_int,total_amount,burden_index,final_employment_length,credit_group,interest_group,loan_group,agg_mean_credit_score_grpby_grade_num,agg_mean_burden_index_grpby_grade_num,agg_mean_credit_score_grpby_employment_length,agg_mean_burden_index_grpby_employment_length,employment_length_purpose_re,employment_length_term_re,purpose_term_re,grade_num_term_re,grade_num_purpose_re,grade_num_employment_length_re,grade_num_re,credit_group_interest_group_re,credit_group_loan_group_re,interest_group_loan_group_re,3 years,5 years,A1,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F3,F5,car,credit_card,debt_consolidation,home_improvement,house,major_purchase,medical,other,small_business,moving
0,0,609.296068,8.421982,0 years,714.061803,0,0,3,0,776.569633,1.087538,3,3.0,0.0,0.0,716.496754,3.176289,677.708997,3.669962,0.214557,0.168913,0.148655,0.032232,0.094642,0.096203,0.080336,0.030183,0.053875,0.039061,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,1,1183.266999,10.286776,10 years,697.706701,0,1,5,10,1930.634031,2.767114,15,2.0,1.0,1.0,683.070367,3.049796,684.668141,4.051284,0.143652,0.312443,0.238716,0.254995,0.121464,0.141902,0.136438,0.129636,0.137194,0.132685,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,2,695.783256,14.723425,1 year,656.419357,0,0,3,1,1050.582789,1.600475,4,0.0,2.0,0.0,674.859374,3.659903,684.394819,3.172965,0.169525,0.136002,0.148655,0.190467,0.211596,0.210969,0.203443,0.178486,0.154018,0.157339,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,3,738.392546,14.260708,0 years,657.906852,0,0,3,0,1101.483602,1.674224,3,0.0,2.0,0.0,674.859374,3.659903,677.708997,3.669962,0.171772,0.168913,0.117253,0.190467,0.184981,0.219450,0.203443,0.178486,0.154018,0.157339,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,4,1642.400654,25.217452,10 years,662.972297,0,0,5,10,5055.957652,7.626197,15,0.0,4.0,2.0,668.965631,7.497618,684.668141,4.051284,0.198372,0.312443,0.308244,0.318968,0.323903,0.330475,0.309276,0.463604,0.259822,0.448159,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0


In [82]:
test.head()

,loan_amnt,interest_rate,credit_score,application_type,term_int,employment_length_int,total_amount,burden_index,final_employment_length,credit_group,interest_group,loan_group,agg_mean_credit_score_grpby_grade_num,agg_mean_burden_index_grpby_grade_num,agg_mean_credit_score_grpby_employment_length,agg_mean_burden_index_grpby_employment_length,employment_length_purpose_re,employment_length_term_re,purpose_term_re,grade_num_term_re,grade_num_purpose_re,grade_num_employment_length_re,grade_num_re,credit_group_interest_group_re,credit_group_loan_group_re,interest_group_loan_group_re,3 years,5 years,A1,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,car,credit_card,debt_consolidation,home_improvement,house,major_purchase,medical,moving,other,small_business,id,employment_length,F3,F5
0,1161.830205,13.105362,682.016083,0,3,10,1681.094955,2.464890,13,1.0,2.0,1.0,674.995781,3.645796,684.758368,4.004564,0.144339,0.143940,0.118036,0.191208,0.185151,0.215389,0.203953,0.184687,0.163279,0.180083,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2257.465815,14.196200,668.240313,0,5,10,4384.089811,6.560649,15,0.0,2.0,3.0,674.995781,3.645796,684.758368,4.004564,0.198207,0.313186,0.310058,0.243651,0.211709,0.215389,0.203953,0.180799,0.297102,0.196001,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2182.137002,8.307772,703.277694,0,3,10,2772.431880,3.942158,13,2.0,0.0,3.0,716.776555,3.133484,684.758368,4.004564,0.144339,0.143940,0.118036,0.032347,0.048908,0.099973,0.080688,0.050680,0.161162,0.046181,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,520.284912,6.956504,731.138779,0,3,7,636.594430,0.870689,10,3.0,0.0,0.0,716.776555,3.133484,686.720668,3.229844,0.151253,0.110251,0.149006,0.032347,0.095222,0.039000,0.080688,0.030683,0.055838,0.039938,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,2303.349549,11.658497,698.368991,0,3,6,3206.529043,4.591454,9,2.0,1.0,3.0,683.275662,3.064383,688.362806,3.952107,0.157433,0.137980,0.149006,0.123396,0.144434,0.128666,0.137705,0.130392,0.161162,0.143296,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [83]:
train.dtypes

id                     int64
loan_amnt            float64
interest_rate        float64
employment_length     object
credit_score         float64
                      ...   
major_purchase         uint8
medical                uint8
other                  uint8
small_business         uint8
moving                 int64
Length: 68, dtype: object

訓練セット，検証セット，テストセットに分割

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, train['loan_status'], test_size=0.2, random_state=0, stratify=train['loan_status'])
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=0, stratify=y_train)

In [85]:
#"顧客id"および目的変数"loan_status"を削除．"employment_length_int"があるため，"employment_length"は不要．
X_train.drop(columns=['employment_length', 'loan_status', 'id'], inplace=True)
X_valid.drop(columns=['employment_length', 'loan_status', 'id'], inplace=True)
X_test.drop(columns=['employment_length', 'loan_status', 'id'], inplace=True)
test.drop(columns=['employment_length', 'id'], inplace=True)


categorical_feature = list(X_train.columns)

In [86]:
X_train.head()

,loan_amnt,interest_rate,credit_score,application_type,term_int,employment_length_int,total_amount,burden_index,final_employment_length,credit_group,interest_group,loan_group,agg_mean_credit_score_grpby_grade_num,agg_mean_burden_index_grpby_grade_num,agg_mean_credit_score_grpby_employment_length,agg_mean_burden_index_grpby_employment_length,employment_length_purpose_re,employment_length_term_re,purpose_term_re,grade_num_term_re,grade_num_purpose_re,grade_num_employment_length_re,grade_num_re,credit_group_interest_group_re,credit_group_loan_group_re,interest_group_loan_group_re,3 years,5 years,A1,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F3,F5,car,credit_card,debt_consolidation,home_improvement,house,major_purchase,medical,other,small_business,moving
180426,1105.658121,10.070675,699.594801,0,3,6,1474.469339,2.107605,9,2.0,1.0,1.0,683.070367,3.049796,687.589733,3.898359,0.129667,0.136708,0.118434,0.123634,0.123107,0.130706,0.138088,0.129691,0.138508,0.134554,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
100977,558.145846,13.825137,677.736756,0,3,3,823.118268,1.214510,6,1.0,2.0,0.0,674.859374,3.659903,685.662516,3.468136,0.162136,0.139376,0.145794,0.190086,0.189801,0.187713,0.203356,0.184362,0.149617,0.158714,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
68638,1138.789953,11.441495,665.532317,0,3,10,1576.102363,2.368183,13,0.0,1.0,1.0,683.070367,3.049796,684.668141,4.051284,0.144645,0.144296,0.118502,0.123393,0.121591,0.141236,0.137807,0.142727,0.193999,0.136538,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
237460,1252.033028,10.767251,698.184045,0,3,10,1701.570401,2.437137,13,2.0,1.0,1.0,683.070367,3.049796,684.668141,4.051284,0.143819,0.144614,0.117873,0.124562,0.123146,0.142696,0.139069,0.133143,0.141806,0.137084,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
94420,999.601028,8.119602,665.481446,0,3,7,1263.397490,1.898471,10,0.0,0.0,1.0,683.070367,3.049796,685.272043,3.323573,0.150535,0.110367,0.149437,0.123393,0.144979,0.098295,0.137807,0.067823,0.193999,0.043367,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [87]:
X_valid.head()

,loan_amnt,interest_rate,credit_score,application_type,term_int,employment_length_int,total_amount,burden_index,final_employment_length,credit_group,interest_group,loan_group,agg_mean_credit_score_grpby_grade_num,agg_mean_burden_index_grpby_grade_num,agg_mean_credit_score_grpby_employment_length,agg_mean_burden_index_grpby_employment_length,employment_length_purpose_re,employment_length_term_re,purpose_term_re,grade_num_term_re,grade_num_purpose_re,grade_num_employment_length_re,grade_num_re,credit_group_interest_group_re,credit_group_loan_group_re,interest_group_loan_group_re,3 years,5 years,A1,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F3,F5,car,credit_card,debt_consolidation,home_improvement,house,major_purchase,medical,other,small_business,moving
195995,1245.110860,14.942785,656.468570,0,5,10,2498.138990,3.805421,15,0.0,2.0,1.0,674.859374,3.659903,684.668141,4.051284,0.199701,0.314830,0.312558,0.243992,0.213416,0.217128,0.205688,0.182397,0.195242,0.181896,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
235915,603.288639,6.902561,793.821840,0,3,3,737.037311,0.928467,6,4.0,0.0,0.0,716.496754,3.176289,685.662516,3.468136,0.185493,0.144985,0.150264,0.031986,0.096471,0.081019,0.080659,0.024349,0.036162,0.040703,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
69069,732.764096,14.406370,656.062704,0,3,10,1097.273268,1.672513,13,0.0,2.0,0.0,674.859374,3.659903,684.668141,4.051284,0.198639,0.144296,0.149437,0.192594,0.213155,0.215972,0.204739,0.181995,0.156791,0.161698,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
24410,3285.403979,13.659911,777.358244,0,5,8,6231.969711,8.016857,13,4.0,2.0,4.0,674.859374,3.659903,684.229075,4.338153,0.188227,0.284462,0.308244,0.243820,0.211596,0.213592,0.203443,0.124528,0.067975,0.209766,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
127954,1064.586551,13.691960,678.574217,0,3,4,1564.480777,2.305541,7,1.0,2.0,1.0,674.859374,3.659903,685.191543,3.664518,0.120996,0.083855,0.147827,0.190086,0.210403,0.130449,0.203356,0.184362,0.163669,0.180691,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [88]:
test.head()

,loan_amnt,interest_rate,credit_score,application_type,term_int,employment_length_int,total_amount,burden_index,final_employment_length,credit_group,interest_group,loan_group,agg_mean_credit_score_grpby_grade_num,agg_mean_burden_index_grpby_grade_num,agg_mean_credit_score_grpby_employment_length,agg_mean_burden_index_grpby_employment_length,employment_length_purpose_re,employment_length_term_re,purpose_term_re,grade_num_term_re,grade_num_purpose_re,grade_num_employment_length_re,grade_num_re,credit_group_interest_group_re,credit_group_loan_group_re,interest_group_loan_group_re,3 years,5 years,A1,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,car,credit_card,debt_consolidation,home_improvement,house,major_purchase,medical,moving,other,small_business,F3,F5
0,1161.830205,13.105362,682.016083,0,3,10,1681.094955,2.464890,13,1.0,2.0,1.0,674.995781,3.645796,684.758368,4.004564,0.144339,0.143940,0.118036,0.191208,0.185151,0.215389,0.203953,0.184687,0.163279,0.180083,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2257.465815,14.196200,668.240313,0,5,10,4384.089811,6.560649,15,0.0,2.0,3.0,674.995781,3.645796,684.758368,4.004564,0.198207,0.313186,0.310058,0.243651,0.211709,0.215389,0.203953,0.180799,0.297102,0.196001,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2182.137002,8.307772,703.277694,0,3,10,2772.431880,3.942158,13,2.0,0.0,3.0,716.776555,3.133484,684.758368,4.004564,0.144339,0.143940,0.118036,0.032347,0.048908,0.099973,0.080688,0.050680,0.161162,0.046181,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,520.284912,6.956504,731.138779,0,3,7,636.594430,0.870689,10,3.0,0.0,0.0,716.776555,3.133484,686.720668,3.229844,0.151253,0.110251,0.149006,0.032347,0.095222,0.039000,0.080688,0.030683,0.055838,0.039938,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,2303.349549,11.658497,698.368991,0,3,6,3206.529043,4.591454,9,2.0,1.0,3.0,683.275662,3.064383,688.362806,3.952107,0.157433,0.137980,0.149006,0.123396,0.144434,0.128666,0.137705,0.130392,0.161162,0.143296,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [91]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_feature)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_feature)

param = {
    'objective': 'binary',
        'metric': 'binary_logloss'
        }

model = lgb.train(param, lgb_train, valid_sets=[lgb_train, lgb_eval], verbose_eval=10, num_boost_round=1000, early_stopping_rounds=100)

y_pred = model.predict(test, num_iteration=model.best_iteration)

[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 25414, number of negative: 119876
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006751 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9225
[LightGBM] [Info] Number of data points in the train set: 145290, number of used features: 51
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.174919 -> initscore=-1.551158
[LightGBM] [Info] Start training from score -1.551158
T

In [92]:
X_pred = model.predict(X_valid, num_iteration=model.best_iteration)
X_pred = (X_pred > 0.18325).astype(int)
f1_score(y_valid, X_pred)

0.37661529450525316

変数重要度を見てみる

In [93]:
pd.set_option('display.max_rows', 100)


# 特徴量重要度の算出方法 'gain'(推奨) : トレーニングデータの損失の減少量を評価
f_importance = np.array(model.feature_importance(importance_type='gain')) # 特徴量重要度の算出
f_importance = f_importance / np.sum(f_importance) # 正規化
df_importance = pd.DataFrame({'feature':categorical_feature, 'importance':f_importance})
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)

,feature,importance
1,interest_rate,0.525830
0,loan_amnt,0.203120
6,total_amount,0.151717
2,credit_score,0.027863
12,agg_mean_credit_score_grpby_grade_num,0.027718
5,employment_length_int,0.014806
4,term_int,0.010555
35,B3,0.006537
7,burden_index,0.005112
47,D5,0.004590


Optunaによるパラメータチューニング

In [94]:
#参考
#https://kiseno-log.com/2019/11/05/lightgbm%E3%81%A8optuna%E3%82%92%E5%B0%8E%E5%85%A5%E3%83%BB%E5%8B%95%E3%81%8B%E3%81%97%E3%81%A6%E3%81%BF%E3%82%8B/

import optuna.integration.lightgbm as lgb_o
from sklearn.model_selection import train_test_split
import sklearn.datasets
from sklearn.metrics import r2_score


# LightGBM用のデータセットに変換
lgb_o_train = lgb_o.Dataset(X_train, y_train)
lgb_o_val = lgb_o.Dataset(X_valid, y_valid)

# ハイパーパラメータサーチ&モデル構築
params = {'objective': 'binary',
          'metric': 'binary_logloss',
          'random_seed':0} 

lgbm_o = lgb_o.train(params,
                    lgb_o_train,
                    valid_sets=lgb_o_val,
                    early_stopping_rounds=100,
                    verbose_eval=200,)

# 調整後モデルで予測の実行
y_valid_pred = lgbm_o.predict(X_valid, num_iteration=lgbm_o.best_iteration)
y_test_pred = lgbm_o.predict(X_test, num_iteration=lgbm_o.best_iteration)

# ベストパラメータの取得
best_params = lgbm_o.params
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))


y_valid_pred = (y_valid_pred > 0.18325).astype(int)
print(f"検証データに対するf1値{f1_score(y_valid, y_valid_pred)}")

y_test_pred = (y_test_pred > 0.18325).astype(int)
print(f"テストデータに対するf1値{f1_score(y_test, y_test_pred)}")

[I 2022-12-10 21:50:21,690] A new study created in memory with name: no-name-8d14c755-5385-433f-9e09-36b0d3801fca
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/Users/tnojima/.pyenv/versions/3.10.1/envs/ml/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/tnojima/.pyenv/versions/3.10.1/envs/ml/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 25414, number of negative: 119876
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004309 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2724
[LightGBM] [Info] Number of data points in the train set: 145290, number of used features: 64
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.174919 -> initscore=-1.551158
[LightGBM] [Info] Start training from score -1.551158
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 0.413095:  14%|#4        | 1/7 [00:01<00:08,  1.47s/it]

[200]	valid_0's binary_logloss: 0.41342
Early stopping, best iteration is:
[115]	valid_0's binary_logloss: 0.413095
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 25414, number of negative: 119876
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005018 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2724
[LightGBM] [Info] Number of data points in the train set: 145290, number of used features: 64
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.174919 -> initscore=-1.551158
[LightGBM] [Info] Start training from score -1.551158
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 0.413095:  29%|##8       | 2/7 [00:02<00:07,  1.46s/it]

...

min_data_in_leaf, val_score: 0.412294:  80%|########  | 4/5 [00:19<00:04,  4.55s/it]

Early stopping, best iteration is:
[654]	valid_0's binary_logloss: 0.412497
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 25414, number of negative: 119876
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005030 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2724
[LightGBM] [Info] Number of data points in the train set: 145290, number of used features: 64
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.174919 -> initscore=-1.551158
[LightGBM] [Info] Start training from score -1.551158
Training until validation scores don't improve for 100 rounds
[200]	valid_0's binary_logloss: 0.413583
[400]	valid_0's binary_logloss: 0.41282
[600]	valid_0's binary_logloss: 0.412534


min_data_in_leaf, val_score: 0.412294: 100%|##########| 5/5 [00:22<00:00,  4.52s/it]

Early stopping, best iteration is:
[655]	valid_0's binary_logloss: 0.412503


  Params: 
    objective: binary
    metric: binary_logloss
    random_seed: 0
    feature_pre_filter: False
    lambda_l1: 3.6503597616135792
    lambda_l2: 0.0003247705016323403
    num_leaves: 5
    feature_fraction: 0.58
    bagging_fraction: 0.7390679027430773
    bagging_freq: 2
    min_child_samples: 50
    num_iterations: 1000
    early_stopping_round: 100
検証データに対するf1値0.3982812161479021
テストデータに対するf1値0.3894258933801992


In [45]:
y_valid_pred = (y_valid_pred > 0.18325).astype(int)
print(f"検証データに対するf1値{f1_score(y_valid, y_valid_pred)}")

y_test_pred = (y_test_pred > 0.18325).astype(int)
print(f"テストデータに対するf1値{f1_score(y_test, y_test_pred)}")

検証データに対するf1値0.4002175883952856
テストデータに対するf1値0.38891739353514615


In [99]:
y_valid_pred_prob = lgbm_o.predict(X_valid, num_iteration=lgbm_o.best_iteration)
y_test_pred_prob = lgbm_o.predict(X_test, num_iteration=lgbm_o.best_iteration)

for threshold in [0.196 + i/10000 for i in range(21)]:
    print(threshold)
    y_valid_pred = (y_valid_pred_prob > threshold).astype(int)
    print(f"検証データに対するf1値{f1_score(y_valid, y_valid_pred)}")

    y_test_pred = (y_test_pred_prob > threshold).astype(int)
    print(f"テストデータに対するf1値{f1_score(y_test, y_test_pred)}")

0.196
検証データに対するf1値0.40007902015013824
テストデータに対するf1値0.38909294155221974
0.1961
検証データに対するf1値0.39996837569672294
テストデータに対するf1値0.3889909722777023
0.1962
検証データに対するf1値0.4000158234107362
テストデータに対するf1値0.389124350259896
0.1963
検証データに対するf1値0.4000950156379904
テストデータに対するf1値0.3893890289280999
0.19640000000000002
検証データに対するf1値0.4000792393026941
テストデータに対するf1値0.38933376040999357
0.1965
検証データに対するf1値0.4001110097926496
テストデータに対するf1値0.38959685821912315
0.1966
検証データに対するf1値0.39999999999999997
テストデータに対するf1値0.38961976576287505
0.1967
検証データに対するf1値0.4000635273564679
テストデータに対するf1値0.3896447922937989
0.1968
検証データに対するf1値0.4000317863954228
テストデータに対するf1値0.389669853000241
0.19690000000000002
検証データに対するf1値0.4001113408620964
テストデータに対するf1値0.38971061093247583
0.197
検証データに対するf1値0.4000954881833373
テストデータに対するf1値0.3898005148005148
0.1971
検証データに対するf1値0.40007960199004977
テストデータに対するf1値0.39003581633063705
0.19720000000000001
検証データに対するf1値0.400031856010831
テストデータに対するf1値0.38988442797889905
0.1973
検証データに対するf1値0.4000637805947541
テストデータに

In [97]:
# 特徴量重要度の算出方法 'gain'(推奨) : トレーニングデータの損失の減少量を評価
f_importance = np.array(lgbm_o.feature_importance(importance_type='gain')) # 特徴量重要度の算出 //
f_importance = f_importance / np.sum(f_importance) # 正規化(必要ない場合はコメントアウト)
df_importance = pd.DataFrame({'feature':categorical_feature, 'importance':f_importance})
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)

,feature,importance
1,interest_rate,0.334224
23,credit_group_interest_group_re,0.151108
25,interest_group_loan_group_re,0.107114
21,grade_num_employment_length_re,0.063809
19,grade_num_term_re,0.055986
16,employment_length_purpose_re,0.036123
2,credit_score,0.031107
20,grade_num_purpose_re,0.029484
17,employment_length_term_re,0.023099
24,credit_group_loan_group_re,0.020848


In [100]:
y_valid_pred_prob = lgbm_o.predict(X_valid, num_iteration=lgbm_o.best_iteration)
y_test_pred_prob = lgbm_o.predict(X_test, num_iteration=lgbm_o.best_iteration)

#LightGBMの出力yに対する閾値の調整．
for threshold in [0.1963 + i/10000 for i in range(21)]:
    print(threshold)
    y_valid_pred = (y_valid_pred_prob > threshold).astype(int)
    print(f"検証データに対するf1値{f1_score(y_valid, y_valid_pred)}")

    y_test_pred = (y_test_pred_prob > threshold).astype(int)
    print(f"テストデータに対するf1値{f1_score(y_test, y_test_pred)}")

0.1963
検証データに対するf1値0.4000950156379904
テストデータに対するf1値0.3893890289280999
0.1964
検証データに対するf1値0.4000792393026941
テストデータに対するf1値0.38933376040999357
0.1965
検証データに対するf1値0.4001110097926496
テストデータに対するf1値0.38959685821912315
0.1966
検証データに対するf1値0.39999999999999997
テストデータに対するf1値0.38961976576287505
0.1967
検証データに対するf1値0.4000635273564679
テストデータに対するf1値0.3896447922937989
0.1968
検証データに対するf1値0.4000317863954228
テストデータに対するf1値0.389669853000241
0.1969
検証データに対するf1値0.4001113408620964
テストデータに対するf1値0.38971061093247583
0.197
検証データに対するf1値0.4000954881833373
テストデータに対するf1値0.3898005148005148
0.1971
検証データに対するf1値0.40007960199004977
テストデータに対するf1値0.39003581633063705
0.19720000000000001
検証データに対するf1値0.400031856010831
テストデータに対するf1値0.38988442797889905
0.1973
検証データに対するf1値0.4000637805947541
テストデータに対するf1値0.3899746141757666
0.1974
検証データに対するf1値0.40003190683205037
テストデータに対するf1値0.3900157264405823
0.1975
検証データに対するf1値0.3999840357598978
テストデータに対するf1値0.3900746419205164
0.1976
検証データに対するf1値0.40022363324148397
テストデータに対するf1値0.38995640238979495

提出のためにtestデータに対して予測を行う．

In [55]:
# 構築したモデルの保存
import datetime

t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)
# YYYYMMDDhhmmss形式に書式化
date = now.strftime('%Y%m%d%H%M%S')



y_pred = lgbm_o.predict(test, num_iteration=lgbm_o.best_iteration)
y_pred = (y_pred > 0.19645).astype(int)
Id = pd.read_csv("new_test.csv").id.astype(int)
my_solution = pd.DataFrame(y_pred, Id, columns=['loan_status'])
my_solution.to_csv(f"my_prediction_data_{date}.csv", header=False)